### How often do we find more parsimonious solutions?

In [1]:
import os
import pickle
from metient.lib.vertex_labeling import *
from metient.util import eval_util as eutil
from metient.util import vertex_labeling_util as vutil

from metient.util.globals import *
from tqdm import tqdm
import fnmatch
import seaborn as sns
import gzip

REPO_DIR = os.path.join(os.getcwd(), "../../")

ModuleNotFoundError: No module named 'metient.lib.vertex_labeling'

In [ ]:
DATE = "multirun_07052024"
PARAMS = "calibrate_wip_solvepoly"
topk = float("inf")
loss_thres = 0.0

SIMS_DIR = os.path.join(REPO_DIR, "data/machina_sims")
metient_results_dir = '/data/morrisq/divyak/data/metient_prediction_results/'

def get_num_mut_trees(mut_tree_fn):
    with open(mut_tree_fn, 'r') as f:
        # look for line w/ "3 #trees" as an example
        for line in f:
            if  "#trees" in line:
                return int(line.strip().split()[0])    
            
def get_pckl_info(bs, run, site, mig_type, seed, tree_num):
    prediction_dir = os.path.join(metient_results_dir, f'predictions_batch_exp_bs{bs*4}_{PARAMS}_r{run}_{DATE}')
    predicted_site_mig_type_data_dir = os.path.join(prediction_dir, site, mig_type)
    with gzip.open(os.path.join(predicted_site_mig_type_data_dir, f"tree{tree_num}_seed{seed}_calibrate.pkl.gz"), "rb") as f:
        pckl = pickle.load(f)
    Vs = pckl[OUT_LABElING_KEY]
    As = pckl[OUT_ADJ_KEY]
    losses = pckl[OUT_LOSSES_KEY]
    return Vs, As, losses


In [ ]:
import torch
og_sims_dir = "/data/morrisq/divyak/projects/machina/data/sims"

def get_V_A_from_ground_truth_tree(site, mig_type, seed):
    labeling_fn = os.path.join(og_sims_dir, site, mig_type, f"T_seed{seed}.vertex.labeling")
    tree_fn = os.path.join(og_sims_dir, site, mig_type, f"T_seed{seed}.tree")
    true_edges, true_mig_edges, true_labeling = eutil.parse_clone_tree(tree_fn, labeling_fn)
    del true_labeling['GL']
    num_nodes = len(true_labeling)
    num_sites = len(set(list(true_labeling.values())))
    node_label_to_idx = {k:i for i,k in enumerate(list(true_labeling.keys()))}
    A = torch.zeros((num_nodes, num_nodes))
    for edge in true_edges:
        if edge[0] == "GL":
            continue
        A[node_label_to_idx[edge[0]], node_label_to_idx[edge[1]]] = 1
    
    site_label_to_idx = {k:i for i,k in enumerate(set(list(true_labeling.values())))}
    V = torch.zeros((num_sites, num_nodes))
    for node_label in true_labeling:
        site_label = true_labeling[node_label]
        V[site_label_to_idx[site_label], node_label_to_idx[node_label]] = 1
    return torch.tensor(V), torch.tensor(A)

def is_more_parsimonious(true_m,true_c,true_s,pred_m,pred_c,pred_s):
    if pred_m > true_m or pred_c > true_c or pred_s > true_s:
        return False
    if pred_m == true_m and pred_c == true_c and pred_s == true_s:
        return False
    return True

In [ ]:
SITES = ["m5", "m8"]
MIG_TYPES = ["mS", "M", "S", "R"]
BATCH_SIZES = ['64', '256', '1024', ]


num_runs = 5
num_less = 0
site_to_bs_to_num_more_pars = {'m5':{bs:[0]*num_runs for bs in BATCH_SIZES}, 'm8':{bs:[0]*num_runs for bs in BATCH_SIZES}}
site_to_bs_to_better_trees = {'m5':{bs:[set()]*num_runs for bs in BATCH_SIZES}, 'm8':{bs:[set()]*num_runs for bs in BATCH_SIZES}}
print(site_to_bs_to_better_trees)
for bs in BATCH_SIZES:
    print(bs)
    for site in SITES:
        for mig_type in MIG_TYPES:
            print(site, mig_type)
            # Get all seeds for mig_type + site combo
            site_mig_type_dir = os.path.join(SIMS_DIR, site, mig_type)
            seeds = fnmatch.filter(os.listdir(site_mig_type_dir), 'clustering_observed_seed*.txt')
            seeds = [s.replace(".txt", "").replace("clustering_observed_seed", "") for s in seeds]
            
            for seed in seeds:
                # Get all the clone trees for this seed
                num_trees = get_num_mut_trees(os.path.join(SIMS_DIR, f"{site}_mut_trees", f"mut_trees_{mig_type}_seed{seed}.txt"))
                
                true_V, true_A = get_V_A_from_ground_truth_tree(site, mig_type, seed)
                true_m,true_c,true_s,_,_ = vutil.ancestral_labeling_metrics(true_V.reshape(1, true_V.shape[0], true_V.shape[1]), true_A, None, None, None, True)
                
                for run in range(1,num_runs+1):
                    for tree_num in range(num_trees):
                        Vs, As, losses = get_pckl_info(bs, run, site, mig_type, seed, tree_num)
                        for x,(V,A,loss) in enumerate(zip(Vs,As,losses)):
                            if x+1 > topk:
                                break
                            V = torch.tensor(V)
                            A = torch.tensor(A)
                            pred_m,pred_c,pred_s,_,_ = vutil.ancestral_labeling_metrics(V.reshape(1, V.shape[0], V.shape[1]), A, None, None, None, True)
                            if is_more_parsimonious(true_m,true_c,true_s,pred_m,pred_c,pred_s):
                                site_to_bs_to_better_trees[site][bs][run-1].add((site,mig_type,seed))
#                                 print("true:", true_m,true_c,true_s)
#                                 print("pred:", pred_m,pred_c,pred_s)
#                                 print()
#                                 print(site_to_bs_to_num_more_pars[site][bs])
                                
                                site_to_bs_to_num_more_pars[site][bs][run-1] += 1
                                break # don't count extra for every V found
site_to_bs_to_num_more_pars


In [ ]:
site_to_bs_to_avg_more_pars = {'m5':{bs:0 for bs in BATCH_SIZES}, 'm8':{bs:0 for bs in BATCH_SIZES}}
for site in site_to_bs_to_num_more_pars:
    for bs in site_to_bs_to_num_more_pars[site]:
        site_to_bs_to_avg_more_pars[site][bs] = sum(site_to_bs_to_num_more_pars[site][bs])/len(site_to_bs_to_num_more_pars[site][bs])
site_to_bs_to_avg_more_pars
        

In [ ]:
site_to_bs_to_better_trees

In [ ]:
site_to_bs_to_avg_better_trees = {'m5':{bs:0 for bs in BATCH_SIZES}, 'm8':{bs:0 for bs in BATCH_SIZES}}
for site in site_to_bs_to_better_trees:
    for bs in site_to_bs_to_better_trees[site]:
        better_trees = site_to_bs_to_better_trees[site][bs]
        average_length = (sum(len(s) for s in better_trees) / len(better_trees))/40.0
        site_to_bs_to_avg_better_trees[site][bs] = average_length
site_to_bs_to_avg_better_trees


In [ ]:
bs_to_better_trees = {bs:[set(), set(), set(), set(), set()] for bs in BATCH_SIZES}
for site in site_to_bs_to_better_trees:
    for bs in site_to_bs_to_better_trees[site]:
        for i,run in enumerate(site_to_bs_to_better_trees[site][bs]):
            for tree in run:
                bs_to_better_trees[bs][i].add(tree)

bs_to_avg_better_trees = {bs:0 for bs in BATCH_SIZES}
for bs in bs_to_better_trees:
    better_trees = bs_to_better_trees[bs]
    average_length = (sum(len(s) for s in better_trees) / len(better_trees))/80.0*100
    bs_to_avg_better_trees[bs] = average_length
bs_to_avg_better_trees


In [ ]:
import pandas as pd
colors = sns.color_palette("crest")
colors = colors[:len(BATCH_SIZES)]
print(colors)
sns.set(font_scale=0.8)
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.set_palette(sns.color_palette(colors))

df = pd.DataFrame(bs_to_avg_better_trees.items(),columns=["Metient sample size", "% of simulated data where more \nparsimonious solution is found"])
print(df)
print(df.columns)
fig = plt.figure(figsize=(3, 3), dpi=300)
plt.ylim(70, 78)  # Adjust the limits as needed

sns.despine()
ax = sns.barplot(x=df["Metient sample size"], y=df["% of simulated data where more \nparsimonious solution is found"])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
# Adjust the width of the bars
bar_width = 0.6  # Adjust this value to make bars thinner or thicker
for bar in ax.patches:
    bar.set_width(bar_width)
fig.savefig(os.path.join(REPO_DIR, "test/output_plots/", f"all_sites_percent_more_pars_{PARAMS}_{DATE}.png"), dpi=600, bbox_inches='tight', pad_inches=0.01) 
